In [1]:
csv_file_path = "./mfi.csv"

In [2]:
!git clone https://github.com/ztuntrade/untrade-sdk.git
!pip install ./untrade-sdk/.

import uuid
import os
from untrade.client import Client

Cloning into 'untrade-sdk'...


Processing c:\users\entro\zelta_final\zelta_final\mfi\untrade-sdk
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5101 sha256=d584298cec0b64be0fa24b7bd902a7704f6bc76b8445d65bb34dc13ae66620e3
  Stored in directory: c:\users\entro\appdata\local\pip\cache\wheels\bc\63\8b\a5da6ae401fa3347590a87bf98ccaa2f71494f073c9bc41b64
Successfully built untrade
  Attempting uninstall: untrade
    Found existing installation: untrade 0.1.0
    Uninstalling untrade-0.1.0:
      Successfully uninstalled untrade-0.1.0


In [3]:
def perform_backtest(csv_file_path):
     client = Client()
     result = client.backtest(
         jupyter_id="test",
         file_path=csv_file_path,
         leverage=1,
     )
     return result


def perform_backtest_large_csv(csv_file_path):
    client = Client()
    file_id = str(uuid.uuid4())
    chunk_size = 90 * 1024 * 1024  # 90 MB
    total_size = os.path.getsize(csv_file_path)
    total_chunks = (total_size + chunk_size - 1) // chunk_size
    chunk_number = 0

    # Handle small files
    if total_size <= chunk_size:
        total_chunks = 1
        result = client.backtest(
            file_path=csv_file_path,
            leverage=1,
            jupyter_id="test",
        )
        for value in result:
            print(value)
        return result

    # Process large files in chunks
    with open(csv_file_path, "rb") as f:
        while True:
            chunk_data = f.read(chunk_size)
            if not chunk_data:
                break

            # Save each chunk temporarily in /tmp
            chunk_file_path = f"/tmp/{file_id}_chunk{chunk_number}.csv"
            with open(chunk_file_path, "wb") as chunk_file:
                chunk_file.write(chunk_data)

            # Perform backtest on the current chunk
            result = client.backtest(
                file_path=chunk_file_path,
                leverage=1,
                jupyter_id="test",
                file_id=file_id,
                chunk_number=chunk_number,
                total_chunks=total_chunks,
            )

            # Process the results of the backtest
            for value in result:
                print(value)

            # Remove the temporary chunk file
            os.remove(chunk_file_path)

            # Move to the next chunk
            chunk_number += 1

    return result

In [4]:
#change to perform_backtest_large_csv(csv_file_path) for large files
backtest_result =  perform_backtest_large_csv(csv_file_path)
print(backtest_result)
for value in backtest_result:
    print(value)

data: {
  "jupyter_id": "test",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 12:00:00",
      "Total Trades": 67,
      "Leverage Applied": 1.0,
      "Winning Trades": 37,
      "Losing Trades": 30,
      "No. of Long Trades": 17,
      "No. of Short Trades": 50,
      "Benchmark Return(%)": 322.726,
      "Benchmark Return(on $1000)": 3227.26,
      "Win Rate": 55.223881,
      "Winning Streak": 5,
      "Losing Streak": 3,
      "Gross Profit": 1196.704725,
      "Net Profit": 1096.204725,
      "Average Profit": 16.361265,
      "Maximum Drawdown(%)": 4.820251,
      "Average Drawdown(%)": 0.903008,
      "Largest Win": 346.191552,
      "Average Win": 40.651643,
      "Largest Loss": -50.817825,
      "Average Loss": -13.596869,
      "Maximum Holding Time": "5 days 0:0:0",
      "Average Holding Time": "1 days 3:13:25",
      "Maximum Adverse Excursion": 5.425967,
      "Average Adverse Excursion": 